In [1]:
!git clone https://github.com/openai/multiagent-particle-envs.git
!pip install gym==0.10.5

Cloning into 'multiagent-particle-envs'...
remote: Enumerating objects: 242, done.
remote: Counting objects: 100% (5/5), done.
remote: Compressing objects: 100% (5/5), done.
remote: Total 242 (delta 0), reused 3 (delta 0), pack-reused 237
Receiving objects: 100% (242/242), 107.24 KiB | 8.94 MiB/s, done.
Resolving deltas: 100% (127/127), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 7.0 MB/s 
     |████████████████████████████████| 966 kB 12.7 MB/s 
  Created wheel for gym: filename=gym-0.10.5-py3-none-any.whl size=1581308 sha256=3f937b9db30052b3fcc5bfb0f370d6e7edc1f77c4c9657989498043a36909643
  Stored in directory: /root/.cache/pip/wheels/7a/2c/df/a05b548a40fae16ca400ecbeda0067e1a296499c1fbd7e0c9a
Successfully built gym
  Attempting uninstall: gym
    Found existing installation: gym 0.25.2
    Uninstalling gym-0.25.2:
      Successfully uninstalled gym-0.25.2


In [2]:
cd multiagent-particle-envs

/content/multiagent-particle-envs


In [3]:
pip install -e.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/multiagent-particle-envs
  Running setup.py develop for multiagent


In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as f
import torch.optim as optim
import numpy as np
from make_env import make_env
env = make_env("simple_tag")
#agent 4 is different agent

In [5]:
class Msg_net(nn.Module):
  def __init__(self):
    super(Msg_net,self).__init__()
    self.fc2= nn.Linear(32,64)
    self.fc3= nn.Linear(64,32)
    self.fc4= nn.Linear(32,2)
  def message_translate(self,state,values,messages,agents):
    w  = torch.cat([state[0],values[0],agents[0]],0)
    v1 = torch.cat([state[1],values[1],agents[1]],0)
    v2 = torch.cat([state[2],values[2],agents[2]],0)
    v  = torch.cat([v1,v2],0)
    v  = torch.rehsape(v,(v.shape[0],1))
    w  = torch.reshape(w,(w.shape[0],1))
    q  = messages
    t1 = torch.matmul(v,q)
    t2 = torch.matmul(w,q)
    final  = t1+t2
    return final  
  def forward(self,states,values,messages):
    final = self.message_translate(states,values,messages)
    fc1   = nn.Linear(final.shape[0],32)
    x     = f.relu(fc1(final))
    x     = f.relu(self.fc2(x))
    x     = f.relu(self.fc3(x))
    x     = f.relu(self.fc4(x))
    return x

In [6]:
class Actor(nn.Module):
  def __init__(self,state_size,action_size,reward_size):
    super(Actor,self).__init__()
    self.state_size = state_size
    self.action_size = action_size
    self.reward_size = reward_size
    self.fc1 = nn.Linear(17,32)
    self.fc2 = nn.Linear(32,64)
    self.fc3 = nn.Linear(64,128)
    self.fc4 = nn.Linear(128,64)
    self.fc5 = nn.Linear(64,32)
    self.fc6 = nn.Linear(32,5)
  def forward(self,state,message):
    x = torch.cat([state,message],0)
    x = f.relu(self.fc1(x))
    x = f.relu(self.fc2(x))
    x = f.relu(self.fc3(x))
    x = f.relu(self.fc4(x))
    x = f.relu(self.fc5(x))
    x = f.softmax(self.fc6(x))
    return x

In [7]:
class Critic(nn.Module):
  def __init__(self,state_size,action_size,reward_size):
    super(Critic,self).__init__()
    self.state_size = state_size
    self.action_size = action_size
    self.reward_size = reward_size
    self.fc1 = nn.Linear(17,32)
    self.fc2 = nn.Linear(32,64)
    self.fc3 = nn.Linear(64,128)
    self.fc4 = nn.Linear(128,64)
    self.fc5 = nn.Linear(64,32)
    self.fc6 = nn.Linear(32,1)
  def forward(self,state):
    x = f.relu(self.fc1(x))
    x = f.relu(self.fc2(x))
    x = f.relu(self.fc3(x))
    x = f.relu(self.fc4(x))
    x = f.relu(self.fc5(x))
    x = f.relu(self.fc6(x))
    return x

In [8]:
class Actor_good(nn.Module):
  def __init__(self,state_size,action_size):
    super(Actor_good,self).__init__()
    self.state_size = state_size
    self.action_size = action_size
    self.fc1 = nn.Linear(14,32)
    self.fc2 = nn.Linear(32,64)
    self.fc3 = nn.Linear(64,128)
    self.fc4 = nn.Linear(128,64)
    self.fc5 = nn.Linear(64,32)
    self.fc6 = nn.Linear(32,5)
  def forward(self,x):
    x = f.relu(self.fc1(x))
    x = f.relu(self.fc2(x))
    x = f.relu(self.fc3(x))
    x = f.relu(self.fc4(x))
    x = f.relu(self.fc5(x))
    x = f.softmax(self.fc6(x))
    return x

In [9]:
class Agent1:
  def __init__(self,state_size,action_size,lr1,lr2,gamma,lamda):
    self.state_size = state_size
    self.action_size = action_size
    self.lr1 = lr1
    self.lr2 = lr2
    self.gamma = gamma
    self.lamda = lamda
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.actor = Actor(self.state_size,self.action).to(self.device)
    self.critic = Critic(self.state_size,self.action_size).to(self.device)
    self.actor_optim = optim.Adam(self.actor.parameters() ,lr = self.lr1)
    self.critic_optim = optim.Adam(self.critic.parameters() ,lr = self.lr2)
  def action(self,state,message):
    action = self.actor(state,message)
    return action
  def q_value(self,state,action,reward):
    value   = torch.cat([state,action,reward],0)
    q_value = self.critic(value)
    return q_value
  def ppo_iters(self,reward,done,value,next_value):
    gae = 0
    returns = []
    for i in range(5):
      delta = reward + next_value * (1-done)*self.gamma - value
      gae   = delta+ gae*self.lamda*(1-done)
      returns.insert(0,gae+value+next_value)
    return returns
  def learn(self,state,next_state,action,next_action,reward,done,message):
    log_prob = torch.log(action).to(self.device)
    next_log_prob = torch.log(next_action).to(self.device)
    ratio = next_log_prob/log_prob
    value = self.q_value(state,action,reward)
    next_value = self.q_value(next_state,next_action,reward)
    returns = self.ppo_iters(reward,done,value,next_value)
    advantage = torch.tensor(returns).float() - value
    s1        = ratio * advantage
    s2        = torch.clamp(ratio ,1- 0.2,0.2)* advantage
    actor_loss = torch.min(s1,s2).mean()
    critic_loss = advantage - value
    critic_loss = critic_loss*critic_loss
    loss        = actor_loss + 0.5*critic_loss 
    self.actor_optim.zero_grad()
    self.critic_optim.zero_grad()
    loss.backward()
    self.actor_optim.step()
    self.critic_optim.step()


In [10]:
class Agent2:
  def __init__(self,state_size,action_size,lr1,lr2,gamma,lamda):
    self.state_size = state_size
    self.action_size = action_size
    self.lr1 = lr1
    self.lr2 = lr2
    self.gamma = gamma
    self.lamda = lamda
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.actor = Actor(self.state_size,self.action).to(self.device)
    self.critic = Critic(self.state_size,self.action_size).to(self.device)
    self.actor_optim = optim.Adam(self.actor.parameters() ,lr = self.lr1)
    self.critic_optim = optim.Adam(self.critic.parameters() ,lr = self.lr2)
  def action(self,state,message):
    action = self.actor(state,message)
    return action
  def q_value(self,state,action,reward):
    value   = torch.cat([state,action,reward],0)
    q_value = self.critic(value)
    return q_value
  def ppo_iters(self,reward,done,value,next_value):
    gae = 0
    returns = []
    for i in range(5):
      delta = reward + next_value * (1-done)*self.gamma - value
      gae   = delta+ gae*self.lamda*(1-done)
      returns.insert(0,gae+value+next_value)
    return returns
  def learn(self,state,next_state,action,next_action,reward,done,message):
    log_prob = torch.log(action).to(self.device)
    next_log_prob = torch.log(next_action).to(self.device)
    ratio = next_log_prob/log_prob
    value = self.q_value(state,action,reward)
    next_value = self.q_value(next_state,next_action,reward)
    returns = self.ppo_iters(reward,done,value,next_value)
    advantage = torch.tensor(returns).float() - value
    s1        = ratio * advantage
    s2        = torch.clamp(ratio ,1- 0.2,0.2)* advantage
    actor_loss = torch.min(s1,s2).mean()
    critic_loss = advantage - value
    critic_loss = critic_loss*critic_loss
    loss        = actor_loss + 0.5*critic_loss 
    self.actor_optim.zero_grad()
    self.critic_optim.zero_grad()
    loss.backward()
    self.actor_optim.step()
    self.critic_optim.step()


In [11]:
class Agent3:
  def __init__(self,state_size,action_size,lr1,lr2,gamma,lamda):
    self.state_size = state_size
    self.action_size = action_size
    self.lr1 = lr1
    self.lr2 = lr2
    self.gamma = gamma
    self.lamda = lamda
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.actor = Actor(self.state_size,self.action).to(self.device)
    self.critic = Critic(self.state_size,self.action_size).to(self.device)
    self.actor_optim = optim.Adam(self.actor.parameters() ,lr = self.lr1)
    self.critic_optim = optim.Adam(self.critic.parameters() ,lr = self.lr2)
  def action(self,state,message):
    action = self.actor(state,message)
    return action
  def q_value(self,state,action,reward):
    value   = torch.cat([state,action,reward],0)
    q_value = self.critic(value)
    return q_value
  def ppo_iters(self,reward,done,value,next_value):
    gae = 0
    returns = []
    for i in range(5):
      delta = reward + next_value * (1-done)*self.gamma - value
      gae   = delta+ gae*self.lamda*(1-done)
      returns.insert(0,gae+value+next_value)
    return returns
  def learn(self,state,next_state,action,next_action,reward,done,message):
    log_prob = torch.log(action).to(self.device)
    next_log_prob = torch.log(next_action).to(self.device)
    ratio = next_log_prob/log_prob
    value = self.q_value(state,action,reward)
    next_value = self.q_value(next_state,next_action,reward)
    returns = self.ppo_iters(reward,done,value,next_value)
    advantage = torch.tensor(returns).float() - value
    s1        = ratio * advantage
    s2        = torch.clamp(ratio ,1- 0.2,0.2)* advantage
    actor_loss = torch.min(s1,s2).mean()
    critic_loss = advantage - value
    critic_loss = critic_loss*critic_loss
    loss        = actor_loss + 0.5*critic_loss 
    self.actor_optim.zero_grad()
    self.critic_optim.zero_grad()
    loss.backward()
    self.actor_optim.step()
    self.critic_optim.step()


In [12]:
class Agent:
  def __init__(self,state_size,action_size,lr1,lr2,gamma,lamda):
    self.state_size = state_size
    self.action_size = action_size
    self.lr1 = lr1
    self.lr2 = lr2
    self.gamma = gamma
    self.lamda = lamda
    self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.actor = Actor_good(self.state_size,self.action).to(self.device)
    self.critic = Critic(self.state_size,self.action_size).to(self.device)
    self.actor_optim = optim.Adam(self.actor.parameters() ,lr = self.lr1)
    self.critic_optim = optim.Adam(self.critic.parameters() ,lr = self.lr2)
  def action(self,state):
    action = self.actor(state)
    return action
  def q_value(self,state,action,reward):
    value   = torch.cat([state,action,reward],0)
    q_value = self.critic(value)
    return q_value
  def ppo_iters(self,reward,done,value,next_value):
    gae = 0
    returns = []
    for i in range(5):
      delta = reward + next_value * (1-done)*self.gamma - value
      gae   = delta+ gae*self.lamda*(1-done)
      returns.insert(0,gae+value+next_value)
    return returns
  def learn(self,state,next_state,action,next_action,reward,done,message):
    log_prob = torch.log(action).to(self.device)
    next_log_prob = torch.log(next_action).to(self.device)
    ratio = next_log_prob/log_prob
    value = self.q_value(state,action,reward)
    next_value = self.q_value(next_state,next_action,reward)
    returns = self.ppo_iters(reward,done,value,next_value)
    advantage = torch.tensor(returns).float() - value
    s1        = ratio * advantage
    s2        = torch.clamp(ratio ,1- 0.2,0.2)* advantage
    actor_loss = torch.min(s1,s2).mean()
    critic_loss = advantage - value
    critic_loss = critic_loss*critic_loss
    loss        = actor_loss + 0.5*critic_loss 
    self.actor_optim.zero_grad()
    self.critic_optim.zero_grad()
    loss.backward()
    self.actor_optim.step()
    self.critic_optim.step()


In [13]:
class Main:
  def __init__(self,state_sizes,actions_sizes,lr1s,lr2s,gamma,lamda):
    self.state_sizes = state_sizes
    self.action_sizes= actions_sizes
    self.lr1s        = lr1s
    self.lr2s        = lr2s
    self.gamma       = gamma
    self.lamda       = lamda
    self.device      = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    self.agent1 = Agent1(self.state_size[0],self.action_size[0],self.lr1[0],self.lr2[0],self.gamma,self.lamda)
    self.agent2 = Agent2(self.state_size[1],self.action_size[1],self.lr1[1],self.lr2[1],self.gamma,self.lamda)
    self.agent3 = Agent3(self.state_size[2],self.action_size[2],self.lr1[2],self.lr2[2],self.gamma,self.lamda)
    self.agent4 = Agent(self.state_size[2],self.action_size[2],self.lr1[2],self.lr2[2],self.gamma,self.lamda)
    self.msg_net= Msg_net().to(self.device())
    self.msg_optim = optim.Adam(self.msg_net.parameters() ,lr = self.lrs[0])
  def choose_actions(self,states,message):
    action1         = self.agent1(states[0],message)
    action2         = self.agent2(states[1],message)
    action3         = self.agent3(states[2],message)
    action4        = self.agent4(states[4])
    actions   = [action1,action2,action3,action4]
    return  actions
  def learn(self,states,next_state,actions,next_actions,rewards,dones,message):
    state1 = torch.tensor(states[0]).float().to(self.device)
    state2 = torch.tensor(states[1]).float().to(self.device)
    state3 = torch.tensor(states[2]).float().to(self.device)
    state4 = torch.tensor(states[3]).float().to(self.device)
    self.agent1.learn(actions[0],next_actions[0],rewards[0],dones[0],message)
    self.agent2.learn(actions[1],next_actions[1],rewards[1],dones[1],message)
    self.agent3.learn(actions[2],next_actions[2],rewards[2],dones[2],message)
    self.agent4.learn(actions[3],next_actions[3],rewards[3],dones[3],message)
  def messages(self,state,actions,values,messages):
    msg_list = self.msg_net(state,actions,values,messages)
    msg = msg_list[0]
    agent = torch.max(msg_list[1])
    if agent==0:
      msgs = [msg,torch.tensor([1]),torch.tensor(1)]
    elif agent==1:
       msgs = [torch.tensor([1]),msg,torch.tensor(1)]
    elif agent==2:
      msgs = [torch.tensor([1]),torch.tensor(1),msg]
    else:
      msgs = [torch.tensor([1]),torch.tensor([1]),torch.tensor(1)]
    return msgs
  def messages_learn(self,state,values,action,messages):
    msgn = self.msg_net(state,values,action,messages)
    idx  = msgn[1]
    
  def play(self,n_games,steps):
    for i in range(n_games,steps):
      state = env.reset()
      done  = [False * 4]
      score = [0,0,0,0]
      for step in range(steps):
        if i==0 and step ==0:
          messages= [torch.tensor([1]),torch.tensor([1]),torch.tensor([1]),torch.tensor([1])]
        else:

        actions = self.choose_actions(state,messages)
        next_state,reward,done,info = env.step(actions)
        next_action  = self.choose_actions(next_state,)
      

IndentationError: ignored